In [40]:
import random
import json

CHARACTER_TILES = {'stone': ' ',
                   'floor': '.',
                   'wall': '#'}


class Gen():
    def __init__(self, width=64, height=64, max_rooms=15, min_room_xy=5, max_room_xy=10):
                 
        self.width = width
        self.height = height
        self.max_rooms = max_rooms
        self.min_room_xy = min_room_xy
        self.max_room_xy = max_room_xy
        self.tiles = CHARACTER_TILES
        self.level = []
        self.room_list = []
        self.corridor_list = []
        self.tiles_level = []

    def gen_room(self):
        #Generate a rectangular room with uniformily distributed parameters:

        #width - between min_room_xy and max_room_xy
        w = random.randint(self.min_room_xy, self.max_room_xy)

        #height - between min_room_xy and max_room_xy
        h = random.randint(self.min_room_xy, self.max_room_xy)

        #x and y coordinates measured at the bottom right corner of the room
        #anywhere in the plane limited between (1,1) and (total width - width of the room - 1 , total height - height of the room - 1)
        x = random.randint(1, (self.width - w - 1))
        y = random.randint(1, (self.height - h - 1))

        return [x, y, w, h]

    def room_overlapping(self, room, room_list):
        x = room[0]
        y = room[1]
        w = room[2]
        h = room[3]

        for current_room in room_list:

            # The rooms don't overlap if one's minimum in some dimension is greater than the other's maximum in that dimension.
            # I'm also adding 3 to every dimension to add a minimun separation of 1 unit between the walls of the rooms 

            if (x < (current_room[0] + current_room[2] + 3) and
                (current_room[0] < (x + w + 3)) and
                (y < (current_room[1] + current_room[3] + 3)) and
                (current_room[1] < (y + h + 3))):

                return True

        return False


    def corridor_between_points(self, x1, y1, x2, y2, join_type='either'):
        
        #if they are in the same row or collumn
        if x1 == x2 or y1 == y2:
            return [(x1, y1), (x2, y2)]

        else:
            # 2 Corridors connected making an L shape, either going vertical-horizontal(top), or horizontal-vertical(bottom)
            # Never randomly choose a join that will go out of bounds when the walls are added.
            join = None

            #check if one of them is at the bottom or left
            if (join_type == 'either') and set([0, 1]).intersection(set([x1, x2, y1, y2])):
                join = 'bottom'

            #check if one of them is at the top or right
            elif ((join_type == 'either') and set([self.width - 1, self.width - 2]).intersection(set([x1, x2]))
                 ) or set([self.height - 1, self.height - 2]).intersection(set([y1, y2])):
                join = 'top'
                
            elif join_type == 'either':
                join = random.choice(['top', 'bottom'])

            else:
                join = join_type

            if join == 'top':
                return [(x1, y1), (x1, y2), (x2, y2)]

            elif join == 'bottom':
                return [(x1, y1), (x2, y1), (x2, y2)]

    def join_rooms(self, room_1, room_2, join_type='either'):
        
        # sort by the value of x
        sorted_room = [room_1, room_2]
        sorted_room.sort(key=lambda x: x[0])

        x1 = sorted_room[0][0]
        y1 = sorted_room[0][1]
        w1 = sorted_room[0][2]
        h1 = sorted_room[0][3]
        x1_2 = x1 + w1 - 1
        y1_2 = y1 + h1 - 1

        x2 = sorted_room[1][0]
        y2 = sorted_room[1][1]
        w2 = sorted_room[1][2]
        h2 = sorted_room[1][3]
        x2_2 = x2 + w2 - 1
        y2_2 = y2 + h2 - 1

        # check for overlapping on x
        if x1 < (x2 + w2) and x2 < (x1 + w1):

            # connect random point on the top wall the bottom room to the bottom wall of the top room
            jx1 = random.randint(x2, x1_2)
            jx2 = jx1
            tmp_y = [y1, y2, y1_2, y2_2]
            tmp_y.sort()
            jy1 = tmp_y[1] + 1
            jy2 = tmp_y[2] - 1
            corridors = self.corridor_between_points(jx1, jy1, jx2, jy2)
            self.corridor_list.append(corridors)

        # check for overlapping on y
        elif y1 < (y2 + h2) and y2 < (y1 + h1):
            if y2 > y1:
                jy1 = random.randint(y2, y1_2)
                jy2 = jy1
            else:
                jy1 = random.randint(y1, y2_2)
                jy2 = jy1

            # connect random point on the right wall of the left room to the left wall of the right room
            tmp_x = [x1, x2, x1_2, x2_2]
            tmp_x.sort()
            jx1 = tmp_x[1] + 1
            jx2 = tmp_x[2] - 1
            corridors = self.corridor_between_points(jx1, jy1, jx2, jy2)
            self.corridor_list.append(corridors)

        # no overlap
        else:
            join = None
            if join_type == 'either':
                join = random.choice(['top', 'bottom'])
            else:
                join = join_type

            if join == 'top':
                #check which room is on top
                if y2 > y1:
                    jx1 = x1_2 + 1
                    jy1 = random.randint(y1, y1_2)
                    jx2 = random.randint(x2, x2_2)
                    jy2 = y2 - 1

                    corridors = self.corridor_between_points(jx1, jy1, jx2, jy2, 'bottom')
                    self.corridor_list.append(corridors)

                else:
                    jx1 = random.randint(x1, x1_2)
                    jy1 = y1 - 1
                    jx2 = x2 - 1
                    jy2 = random.randint(y2, y2_2)

                    corridors = self.corridor_between_points(jx1, jy1, jx2, jy2, 'top')
                    self.corridor_list.append(corridors)

            elif join == 'bottom':
                #check which room is on top
                if y2 > y1:
                    jx1 = random.randint(x1, x1_2)
                    jy1 = y1_2 + 1
                    jx2 = x2 - 1
                    jy2 = random.randint(y2, y2_2)

                    corridors = self.corridor_between_points(jx1, jy1, jx2, jy2, 'top')
                    self.corridor_list.append(corridors)

                else:
                    jx1 = x1_2 + 1
                    jy1 = random.randint(y1, y1_2)
                    jx2 = random.randint(x2, x2_2)
                    jy2 = y2_2 + 1

                    corridors = self.corridor_between_points(jx1, jy1, jx2, jy2, 'bottom')
                    self.corridor_list.append(corridors)


    def gen_level(self):

        # build an empty dungeon, blank the room and corridor lists
        for i in range(self.height):
            self.level.append(['stone'] * self.width)
        self.room_list = []
        self.corridor_list = []

        #set a limit if the dungeon does not hit the max_rooms number
        max_iters = self.max_rooms * 5  

        temp_room_list = []
        for _ in range(max_iters):
            #creates a new room
            tmp_room = self.gen_room()

            #check if it is not overlapping
            if not self.room_overlapping(tmp_room, temp_room_list):
                #add to the temporary room list
                temp_room_list.append(tmp_room)

            #if it hits the max number of rooms
            if len(temp_room_list) >= self.max_rooms:
                break

        #select 2 random rooms from the temporary list
        #connect both
        #randomly select one of the 2 selected rooms and move it from the temp list to the main list
        #repeat untill there is only 1 room left in the temporary list
        while len(temp_room_list) > 1:
            index_rooms_to_connect = random.sample(range(len(temp_room_list)),k=2)
            self.join_rooms(temp_room_list[index_rooms_to_connect[0]], temp_room_list[index_rooms_to_connect[1]])
            if random.random() > 0.5:
                self.room_list.append(temp_room_list.pop(index_rooms_to_connect[0]))
            else:
                self.room_list.append(temp_room_list.pop(index_rooms_to_connect[1]))
        self.room_list.append(temp_room_list.pop())

        # fill the map
        # paint rooms
        for room_num, room in enumerate(self.room_list):
            for i in range(room[2]):
                for j in range(room[3]):
                    self.level[room[1] + j][room[0] + i] = 'floor'

        # paint corridors
        for corridor in self.corridor_list:
            x1, y1 = corridor[0]
            x2, y2 = corridor[1]
            for width in range(abs(x1 - x2) + 1):
                for height in range(abs(y1 - y2) + 1):
                    self.level[min(y1, y2) + height][
                        min(x1, x2) + width] = 'floor'

            if len(corridor) == 3:
                x3, y3 = corridor[2]

                for width in range(abs(x2 - x3) + 1):
                    for height in range(abs(y2 - y3) + 1):
                        self.level[min(y2, y3) + height][
                            min(x2, x3) + width] = 'floor'

        # paint the walls
        for row in range(1, self.height - 1):
            for col in range(1, self.width - 1):
                if self.level[row][col] == 'floor':
                    if self.level[row - 1][col - 1] == 'stone':
                        self.level[row - 1][col - 1] = 'wall'

                    if self.level[row - 1][col] == 'stone':
                        self.level[row - 1][col] = 'wall'

                    if self.level[row - 1][col + 1] == 'stone':
                        self.level[row - 1][col + 1] = 'wall'

                    if self.level[row][col - 1] == 'stone':
                        self.level[row][col - 1] = 'wall'

                    if self.level[row][col + 1] == 'stone':
                        self.level[row][col + 1] = 'wall'

                    if self.level[row + 1][col - 1] == 'stone':
                        self.level[row + 1][col - 1] = 'wall'

                    if self.level[row + 1][col] == 'stone':
                        self.level[row + 1][col] = 'wall'

                    if self.level[row + 1][col + 1] == 'stone':
                        self.level[row + 1][col + 1] = 'wall'

    def gen_tiles_level(self):

        for row in self.level:
            tmp_tiles = []

            for col in row:
                if col == 'stone':
                    tmp_tiles.append(self.tiles['stone'])
                if col == 'floor':
                    tmp_tiles.append(self.tiles['floor'])
                if col == 'wall':
                    tmp_tiles.append(self.tiles['wall'])

            self.tiles_level.append(''.join(tmp_tiles))

        [print(row) for row in self.tiles_level]
        

    def rowParm(self):

        for row in self.level:
            tmp_tiles = []

            for col in row:
                if col == 'stone':
                    tmp_tiles.append(self.tiles['stone'])
                if col == 'floor':
                    tmp_tiles.append(self.tiles['floor'])
                if col == 'wall':
                    tmp_tiles.append(self.tiles['wall'])

            self.tiles_level.append(''.join(tmp_tiles))

        rows = []
        for row in self.tiles_level:
            c = 0
            r = []
            prev = row[0]
            for (i,cu) in enumerate(row):
                c += 1
                if prev != cu or i==len(row)-1:
                    r.append((c,prev))
                    c = 0
                prev = cu
            rows.append(r)
        
        return rows
        

    def toJ(self):

        TILES = {' ' : 'stone',
                '.' : 'floor',
                '#' : 'wall'}


        rows = []

        for row in self.rowParm():
            rows.append( 
                {"Row" :
                    [ 
                        {
                        "Type" : TILES[row[i][1]],
                        "Count" : row[i][0]
                        } 
                        for i in range(len(row))
                    ] 
                }
            )
            

        j = {
            "Height" : self.height,
            "Width" : self.width,
            "Rows" : rows
        }

        return json.dumps(j, indent=4)



In [42]:
if __name__ == '__main__':
    level = Gen(width=50, height=50, max_rooms=5, min_room_xy=3, max_room_xy=10)
    level.gen_level()
    print(level.toJ())
    with open('out.json', 'w') as outfile:
        outfile.write(level.toJ())

{
    "Height": 50,
    "Width": 50,
    "Rows": [
        {
            "Row": [
                {
                    "Type": "stone",
                    "Count": 50
                }
            ]
        },
        {
            "Row": [
                {
                    "Type": "stone",
                    "Count": 50
                }
            ]
        },
        {
            "Row": [
                {
                    "Type": "stone",
                    "Count": 50
                }
            ]
        },
        {
            "Row": [
                {
                    "Type": "stone",
                    "Count": 50
                }
            ]
        },
        {
            "Row": [
                {
                    "Type": "stone",
                    "Count": 50
                }
            ]
        },
        {
            "Row": [
                {
                    "Type": "stone",
                    "Count": 50
                }
        

' '